In [1]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
import dbfread 
import geopandas as gpd
import warnings
import matplotlib.pyplot as plt
import csv
import pyproj
from simpledbf import Dbf5
from datetime import datetime

In [2]:
# crotia, harmonized dataset from MultiOne
crotia = pd.read_excel('/mnt/primus/xuemeng_tmp_harbour/soc_eu/crotia/hr_topsoil_db.xlsx')

# filter/organize based on depth
na = crotia[crotia['source_db']=='martinovic_1997']['dbr'].isna().sum()
print(f'- {na} data, for no depth info')
crotia = crotia[~((crotia['source_db'] == 'martinovic_1997') & (crotia['dbr'].isna()))]  # without any depth information, leave out
crotia['hzn_top'] = 0
crotia['hzn_btm'] = 30
crotia.loc[crotia['source_db'] == 'martinovic_1997', 'hzn_top'] = crotia.loc[crotia['source_db'] == 'martinovic_1997', 'dbr'] - 5
crotia.loc[crotia['source_db'] == 'martinovic_1997', 'hzn_btm'] = crotia.loc[crotia['source_db'] == 'martinovic_1997', 'dbr'] + 5
crotia.loc[crotia['source_db'] == 'azo_2013', 'hzn_btm'] = 25


column_names = ['ph_h2o','ph_ca','oc','gps_lat','gps_long','time','hzn_top','hzn_btm','ref','ph_kcl']
temp = pd.DataFrame(columns=column_names)
temp['oc'] = crotia['oc']*10
temp['ph_kcl'] = crotia['ph_kcl']
temp['ph_h2o'] = crotia['ph_h2o']
temp['ph_ca'] = np.nan
temp['ref'] = crotia['source_db']+'_MultiOne'
temp['gps_lat'] = crotia['latitude_decimal_degrees']
temp['gps_long'] = crotia['longitude_decimal_degrees']
temp['country'] = 'crotia'
temp['time'] = crotia['site_obsdate']
temp['hzn_top'] = crotia['hzn_top']
temp['hzn_btm'] = crotia['hzn_btm']
# df = pd.concat([df,temp])

na = temp['time'].isna().sum()
print(f'- {na} data, for no time info')
temp = temp.dropna(subset=['time'])

na = len(temp[temp['gps_lat'].isna() | temp['gps_long'].isna()])
print(f'- {na} data, for no coordinate info')
temp = temp.dropna(subset=['gps_lat','gps_long'])

print(f'{len(temp)} data left in crotia')
temp.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc_eu/data/training_point_v2_crotia.csv',index=False)

- 339 data, for no depth info
- 2520 data, for no time info
- 0 data, for no coordinate info
3412 data left in crotia


In [3]:
# germany
germany = pd.read_excel(r'/mnt/diskstation/data/soil_points/Germany/LABORATORY_DATA.xlsx', engine='openpyxl')
germany_site = pd.read_excel(r'/mnt/diskstation/data/Soil_points/Germany/SITE.xlsx', engine='openpyxl')
germany = germany.merge(germany_site, on="PointID", how="inner")
utm_projection = pyproj.CRS.from_string(f'+proj=utm +zone={32} +ellps=WGS84')
gps_projection = pyproj.CRS.from_epsg(4326)
# Create transformer objects for the coordinate conversion
transformer = pyproj.Transformer.from_crs(utm_projection, gps_projection)
# Convert UTM coordinates to GPS latitude and longitude
germany['lat'], germany['lon'] = transformer.transform(germany['xcoord'], germany['ycoord'])
column_names = ['ph_h2o','ph_ca','oc','gps_lat','gps_long','time','hzn_top','hzn_btm','ref','ph_kcl']
temp = pd.DataFrame(columns=column_names)
temp['oc'] = germany['TOC']
temp['ph_kcl'] = np.nan
temp['ph_h2o'] = germany['pH_H2O']
temp['ph_ca'] = germany['pH_CaCl2']
temp['time'] = germany['Sampling_year']
temp['hzn_top'] = germany['Layer upper limit']
temp['hzn_btm'] = germany['Layer lower limit']
temp['ref'] = 'https://literatur.thuenen.de/digbib_extern/dn062722.pdf'
temp['gps_lat'] = germany['lat']
temp['gps_long'] = germany['lon']
temp['country'] = 'germany'

na = temp['time'].isna().sum()
print(f'- {na} data, for no time info')
temp = temp.dropna(subset=['time'])

na = len(temp[temp['gps_lat'].isna() | temp['gps_long'].isna()])
print(f'- {na} data, for no coordinate info')
temp = temp.dropna(subset=['gps_lat','gps_long'])

na = len(temp[temp['hzn_btm'].isna() | temp['hzn_top'].isna()])
print(f'- {na} data, for no depth info')
temp = temp.dropna(subset=['hzn_top','hzn_btm'])

print(f'{len(temp)} data left in germany')
temp.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc_eu/data/training_point_v2_germany.csv',index=False)

- 0 data, for no time info
- 0 data, for no coordinate info
- 0 data, for no depth info
17189 data left in germany


In [9]:
# +belgium
# read in 2 sites
belgium_p = pd.read_csv('/mnt/diskstation/data/soil_points/Belgium/Vlaanderen/Aardewerk-Vlaanderen-2010_Profiel.csv')
belgium_h = pd.read_csv('/mnt/diskstation/data/soil_points/Belgium/Vlaanderen/Aardewerk-Vlaanderen-2010_Horizont.csv',low_memory=False,encoding = "ISO-8859-1")
# merge 2 sites
belgium_p = belgium_p.rename(columns={'ID': 'Profiel_ID'}) 
belgium = belgium_h.merge(belgium_p, on="Profiel_ID", how="inner")
# Define the coordinate systems
lambert72 = pyproj.CRS.from_epsg(31370)  # Lambert72 CRS
wgs84 = pyproj.CRS.from_epsg(4326)  # WGS84 CRS (GPS)
transformer = pyproj.Transformer.from_crs(lambert72, wgs84)
belgium['lat'], belgium['lon'] = transformer.transform(belgium['Coordinaat_Lambert72_X'], belgium['Coordinaat_Lambert72_Y'])
# belgium['Y'], belgium['X'] = transformer.transform(belgium['Coordinaat_Bonne_E'], belgium['Coordinaat_Bonne_N'])

# convert humus to oc
belgium.loc[belgium['Humus_koolstof_nieuwe_formule']==0, 'Humus'] = belgium.loc[belgium['Humus_koolstof_nieuwe_formule']==0, 'Humus'] / 1.724
belgium.loc[belgium['Humus_koolstof_nieuwe_formule']==1, 'Humus'] = belgium.loc[belgium['Humus_koolstof_nieuwe_formule']==1, 'Humus'] / 2

# extract time info 
na = belgium['Profilering_Datum'].isna().sum()
print(f'- {na} data, for no time info')
belgium = belgium.dropna(subset=['Profilering_Datum'])
belgium['Profilering_Datum'] = belgium['Profilering_Datum'].str.split(' ').str[0]
belgium['Profilering_Datum'] = belgium['Profilering_Datum'].str.split('-').str[-1].astype(int)
na = len(belgium.loc[belgium['Profilering_Datum']==2094])
print(f'- {na} data, for invalid time info')
belgium = belgium[belgium['Profilering_Datum'] != 2094]

# extract depth info
belgium['hzn_top'] = np.nanmin(belgium[['Diepte_grens_boven1', 'Diepte_grens_boven2']], axis=1)
belgium['hzn_btm'] = np.nanmax(belgium[['Diepte_grens_onder1', 'Diepte_grens_onder2']], axis=1)

na = len(belgium.loc[belgium['hzn_top']>=belgium['hzn_btm']])
print(f'- {na} data, for no valid depth info')
belgium = belgium.loc[belgium['hzn_top'] < belgium['hzn_btm']]

na = len(belgium[belgium['hzn_btm'].isna() | belgium['hzn_top'].isna()])
print(f'- {na} data, for no depth info')
belgium = belgium.dropna(subset=['hzn_top','hzn_btm'], how='any')

# merge belgium data into lucas
column_names = ['ph_h2o','ph_ca','oc','gps_lat','gps_long','time','hzn_top','hzn_btm','ref','ph_kcl']
temp = pd.DataFrame(columns=column_names)
temp['oc'] = belgium['Humus']*10
temp['ph_kcl'] = belgium['pH_KCl']
temp['ph_h2o'] = belgium['pH_H2O']
temp['ph_ca'] = np.nan
temp['time'] = belgium['Profilering_Datum']
temp['hzn_top'] = belgium['Diepte_grens_boven1']
temp['hzn_btm'] = belgium['Diepte_grens_onder1']
temp.loc[temp['hzn_top'].isna(),'hzn_top'] = belgium.loc[temp['hzn_top'].isna(),'Diepte_grens_boven2']
temp.loc[temp['hzn_btm'].isna(),'hzn_btm'] = belgium.loc[temp['hzn_btm'].isna(),'Diepte_grens_onder2']
temp['hzn_top'] = np.nanmax(belgium[['Diepte_grens_boven1', 'Diepte_grens_boven2']], axis=1)
temp['hzn_btm'] = np.nanmax(belgium[['Diepte_grens_onder1', 'Diepte_grens_onder2']], axis=1)     
temp['gps_lat'] = belgium['lat']
temp['gps_long'] = belgium['lon']

# filter based on coordinates
na = len(temp[temp['gps_lat'].isna() | temp['gps_long'].isna()])
print(f'- {na} data, for no coordinate info')
temp = temp.dropna(subset=['gps_lat','gps_long'])

# merge duplicates
dupm = temp.groupby(['gps_lat', 'gps_long', 'time','hzn_top','hzn_btm'])[['oc', 'ph_h2o','ph_kcl','ph_ca']].mean().reset_index()
dupm['ref'] = 'https://www.dov.vlaanderen.be/geonetwork/srv/api/records/78e15dd4-8070-4220-afac-258ea040fb30'
dupm['country'] = 'belgium'
print(f'- {len(temp)-len(dupm)} data, for duplicate time, coordinate, and depth info')

print(f'{len(dupm)} data left in belgium')
dupm.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc_eu/data/training_point_v2_belgium.csv',index=False)

- 3536 data, for no time info
- 5 data, for invalid time info
- 224 data, for no valid depth info
- 0 data, for no depth info
- 28 data, for no coordinate info
- 514 data, for duplicate time, coordinate, and depth info
36309 data left in belgium


/tmp/ipykernel_151658/3479290969.py:30: RuntimeWarning: All-NaN axis encountered
  belgium['hzn_top'] = np.nanmin(belgium[['Diepte_grens_boven1', 'Diepte_grens_boven2']], axis=1)
/tmp/ipykernel_151658/3479290969.py:31: RuntimeWarning: All-NaN axis encountered
  belgium['hzn_btm'] = np.nanmax(belgium[['Diepte_grens_onder1', 'Diepte_grens_onder2']], axis=1)


In [5]:
# scotland
scotland = pd.read_excel('/mnt/diskstation/data/soil_points/Scotland/NSIS_1_10km_grid_gh.xlsx', sheet_name='NSIS1_10km')
osgb36 = pyproj.CRS.from_string("+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=400000 +y_0=-100000 +ellps=airy +towgs84=446.448,-125.157,542.060,0.1502,0.2470,0.8421,-20.4894 +units=m +no_defs")
wgs84 = pyproj.CRS.from_epsg(4326)
transformer = pyproj.Transformer.from_crs(osgb36, wgs84)
scotland['lat'], scotland['lon'] = transformer.transform(scotland['EASTING'], scotland['NORTHING'])
column_names = ['ph_h2o','ph_ca','oc','gps_lat','gps_long','time','hzn_top','hzn_btm','ref','ph_kcl']
temp = pd.DataFrame(columns=column_names)
temp['oc'] = scotland['DP1971_ORGANIC_MATTER']/1.72
temp['ph_kcl'] = np.nan
temp['ph_h2o'] = scotland['DP1971_PH_H2O']
temp['ph_ca'] = scotland['DP1971_PH_CACL2']
temp['ref'] = 'https://www.hutton.ac.uk/about/facilities/national-soils-archive/resampling-soils-inventory'
temp['gps_lat'] = scotland['lat']
temp['gps_long'] = scotland['lon']
temp['country'] = 'scotland'
temp['time'] = scotland['PROFILE_DATE'].astype(str).str[-4:]
temp['hzn_top'] = scotland['HORZ_TOP']
temp['hzn_btm'] = scotland['HORZ_BOTTOM']

na = temp['time'].isna().sum()
print(f'- {na} data, for no time info')
temp = temp.dropna(subset=['time'])

na = len(temp[temp['gps_lat'].isna() | temp['gps_long'].isna()])
print(f'- {na} data, for no coordinate info')
temp = temp.dropna(subset=['gps_lat','gps_long'])

na = len(temp[temp['hzn_btm'].isna() | temp['hzn_top'].isna()])
print(f'- {na} data, for no depth info')
temp = temp.dropna(subset=['hzn_top','hzn_btm'])

print(f'{len(temp)} data left in scotland')
temp.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc_eu/data/training_point_v2_scotland.csv',index=False)

- 0 data, for no time info
- 0 data, for no coordinate info
- 86 data, for no depth info
3005 data left in scotland
